"FC layers referenced from https://towardsdatascience.com/math-neural-network-from-scratch-in-python-d6da9f29ce65"


In [36]:
import torch
import os
import numpy as np
import pickle
import numpy as np
import torch.nn as nn
import torch.nn.functional as F
from torch.utils.data import DataLoader, TensorDataset, Dataset
import torch.optim as optim
from sklearn.metrics import precision_score, recall_score, f1_score, precision_recall_fscore_support
import matplotlib.pyplot as plt
from sklearn.metrics.pairwise import cosine_similarity
from sklearn.datasets import make_classification
from sklearn.model_selection import train_test_split
from sklearn.tree import DecisionTreeClassifier
from sklearn.metrics import accuracy_score
from sklearn.feature_selection import SelectKBest, f_classif, chi2
from sklearn.decomposition import PCA
from sklearn.preprocessing import MinMaxScaler
import plotly.graph_objs as go
from sklearn.manifold import TSNE
import plotly.io as pio
from sklearn.utils import class_weight
import tqdm as notebook_tqdm
from tqdm import tqdm

In [3]:
# !pip install ipywidgets

<h3> Declare functions

In [37]:
class FCLayer(nn.Module):
    def __init__(self, input_dim, output_dim):
        super(FCLayer, self).__init__()
        self.fc = nn.Linear(input_dim, output_dim)

    def forward(self, x):
        x = self.fc(x)
        return x

class ActivationLayer(nn.Module):
    def __init__(self, activation_fn):
        super(ActivationLayer, self).__init__()
        self.activation_fn = activation_fn

    def forward(self, x):
        x = self.activation_fn(x)
        return x

def tanh(x):
    return torch.tanh(x)

def sigmoid(x):
    return torch.sigmoid(x)

class MyNetwork(nn.Module):
    def __init__(self, input_dim, hidden_dim, output_dim):
        super(MyNetwork, self).__init__()
        self.fc1 = FCLayer(input_dim, hidden_dim)
        self.activation1 = ActivationLayer(tanh)
        self.fc2 = FCLayer(hidden_dim, output_dim)
        self.activation2 = ActivationLayer(sigmoid)

    def forward(self, x):
        x = self.fc1(x)
        x = self.activation1(x)
        x = self.fc2(x)
        x = self.activation2(x)
        return x

# loss function and its derivative
def mse(y_true, y_pred):
    return np.mean(np.power(y_true - y_pred, 2))

def mse_prime(y_true, y_pred):
    return 2 * (y_pred - y_true) / y_true.size


In [60]:
# Function to balance class distribution using oversampling
def oversample_data(X_train, Y_train, num_classes):
    # Determine the class with the maximum number of instances
    max_class_count = np.max(np.bincount(Y_train))
    # Generate indices for oversampling each class
    indices_list = [np.where(Y_train == i)[0] for i in range(num_classes)]
    # Oversample minority classes to match the count of the majority class
    for i, indices in enumerate(indices_list):
        if len(indices) < max_class_count:
            oversampled_indices = np.random.choice(indices, size=max_class_count - len(indices), replace=True)
            X_train = np.concatenate((X_train, X_train[oversampled_indices]), axis=0)
            Y_train = np.concatenate((Y_train, Y_train[oversampled_indices]), axis=0)
    return X_train, Y_train


<h4> Extract train labels and label encoder

In [39]:
file_path = "data/dump/labels_train.pkl"
with open(file_path, 'rb') as file:
    y_train = pickle.load(file)
y_train = torch.tensor(y_train)
    
file_path = 'data/dump/label_decoder.pkl'
with open(file_path, 'rb') as file:
    label_decoder = pickle.load(file)

<h4> Extract test labels

In [40]:
file_path = "data/dump/labels_test.pkl"
with open(file_path, 'rb') as file:
    file = open('data/dump/labels_test.pkl', 'rb')
    y_test = pickle.load(file)
y_test = torch.tensor(y_test)

<h4> Getting BERT and GAT outputs for train

In [41]:
file_path = 'embed/u_prime_BERT_train.pkl'
with open(file_path, 'rb') as file:
    u_primes = pickle.load(file)

    concatenated_tensors = []
    for dialogue_tensor in u_primes:
        concatenated_tensors.extend(dialogue_tensor)

    tensorUtterancesTrain = torch.stack(concatenated_tensors)

file_path = "data/dump/h_prime_BERT-GAT_train.pkl"
with open(file_path, 'rb') as file:
    cherryPickedNodesTrain, _ = pickle.load(file)

file_path = "data/dump/h_prime_BERT-EGAT_train.pkl"
with open(file_path, 'rb') as file:
    allNodeFeatsTrain, _ = pickle.load(file)

<h4> Getting BERT and GAT outputs for test set

In [42]:
file_path = 'embed/u_prime_BERT_test.pkl'
with open(file_path, 'rb') as file:
    u_primes = pickle.load(file)

    concatenated_tensors = []
    for dialogue_tensor in u_primes:
        concatenated_tensors.extend(dialogue_tensor)

    tensorUtterancesTest = torch.stack(concatenated_tensors)

file_path = "data/dump/h_prime_BERT-GAT_test.pkl"
with open(file_path, 'rb') as file:
    cherryPickedNodesTest, _ = pickle.load(file)

file_path = "data/dump/h_prime_BERT-EGAT_test.pkl"
with open(file_path, 'rb') as file:
    allNodeFeatsTest, _ = pickle.load(file)
    
_ = None

<h4> Getting BERT and GAT outputs for the valid set

In [ ]:
# TODO do the same code as above once you have u' and h' of valid set

EDA

In [8]:
# # Checking the structure of graph
# for n in range(10):
#     tensor_data_np = tensor_utterances[n].detach().numpy()

#     # Plot the data
#     plt.figure(figsize=(10, 5))
#     plt.plot(range(len(tensor_data_np)), tensor_data_np)
#     plt.title('Line Graph of Tensor Data')
#     plt.xlabel('Index')
#     plt.ylabel('Value')
#     plt.show()


In [9]:
# # Normalize the h' (1st GAT)
# data = cherry_picked_nodes.detach().numpy()
# data_normalized = data / np.linalg.norm(data, axis=1, keepdims=True)

# # Compute pairwise cosine similarities
# similarities = cosine_similarity(data_normalized)

# # Print or analyze the similarity matrix
# # print(similarities)
# plt.hist(similarities.flatten(), bins=50, density=True)
# plt.title('Distribution of Cosine Similarities')
# plt.xlabel('Cosine Similarity')
# plt.ylabel('Frequency')
# plt.show()


In [10]:
# # Normalize the h' (2nd GAT)
# data = all_node_feats.detach().numpy()
# data_normalized = data / np.linalg.norm(data, axis=1, keepdims=True)

# # Compute pairwise cosine similarities
# similarities = cosine_similarity(data_normalized)

# # Print or analyze the similarity matrix
# # print(similarities)
# plt.hist(similarities.flatten(), bins=50, density=True)
# plt.title('Distribution of Cosine Similarities')
# plt.xlabel('Cosine Similarity')
# plt.ylabel('Frequency')
# plt.show()


In [11]:
# # Normalize the u' or updated_representations
# data = tensor_utterances.detach().numpy()
# data_normalized = data / np.linalg.norm(data, axis=1, keepdims=True)

# # Compute pairwise cosine similarities
# similarities = cosine_similarity(data_normalized)

# plt.hist(similarities.flatten(), bins=50, density=True)
# plt.title('Distribution of Cosine Similarities')
# plt.xlabel('Cosine Similarity')
# plt.ylabel('Frequency')
# plt.show()


<h3> Feature Selection

In [12]:
# X_train = tensor_utterances
# Y_train = y_train

Part 1

In [13]:
# # Define the number of features (k) to select
# k = 100  # Adjust this value as needed

# # Initialize SelectKBest with the desired score function (e.g., f_classif for classification tasks)
# selector = SelectKBest(score_func=f_classif, k=k)

# # Fit SelectKBest on the training data and target variable
# selector.fit(X_train, Y_train)

# # Get the indices of the selected features
# selected_indices = selector.get_support(indices=True)

# # Get the scores of the selected features
# feature_scores = selector.scores_[selected_indices]

# # Display the scores along with their corresponding indices
# # for idx, score in zip(selected_indices, feature_scores):
# #     print(f"Feature index: {idx}, Score: {score}")

# X_train_selected = X_train[:, selected_indices]
# print(X_train_selected.shape)

Pass u' (BERT) and h' (GAT or EGAT) into this method

In [86]:
def get_selected_features(encoded_features, labels, top_n):
    # Apply Min-Max scaling to make the data non-negative
    scaler = MinMaxScaler()
    features_scaled = scaler.fit_transform(encoded_features)

    # Initialize SelectKBest with the desired score function (e.g., f_classif for classification tasks)
    selector = SelectKBest(score_func=f_classif, k=100)
    # Assuming feature is your feature matrix (12840 instances x 300 dimensions)
    # and y_train is your target labels

    # Initialize a dictionary to store the indices of top features for each class
    top_features_by_class = {}
    top_scores = {}
    # Calculate the relevance of each feature to each class using chi-squared test
    for label in range(7):  # Assuming you have 7 classes
        # Create a binary mask indicating instances belonging to the current class
        mask = (labels == label)

        # SelectKBest with chi2 as the scoring function
        selector = SelectKBest(score_func=chi2, k=top_n)  # Select top 20 features
        selector.fit(features_scaled, mask)  # Fit SelectKBest to the data
        # Get the indices of the top 20 features
        top_features_indices = np.argsort(selector.scores_)[-top_n:]
        scores = selector.scores_[top_features_indices]
        # Store the indices in the dictionary
        top_features_by_class[label] = top_features_indices
        top_scores[label] = scores

    # Print the top features for each class
    # for label, indices in top_features_by_class.items():
    #     print(f"Label {label_decoder[label]}: idx {', '.join(map(str, indices))}")
    #     print(top_scores[label])

    concatenated_features_set = set()
    for label, indices in top_features_by_class.items():
        concatenated_features_set.update(indices)

    concatenated_features_indices = list(concatenated_features_set)

    concatenated_features_indices = np.array(concatenated_features_indices)

    # Select the desired features
    selected_features = encoded_features[:, concatenated_features_indices]
#     print(selected_features.shape)
    return selected_features, concatenated_features_indices

Selected h'

In [17]:
# X_train = all_node_feats

In [35]:
# # Apply Min-Max scaling to make the data non-negative
# scaler = MinMaxScaler()
# X_train_scaled = scaler.fit_transform(X_train)

# # Initialize SelectKBest with the desired score function (e.g., f_classif for classification tasks)
# selector = SelectKBest(score_func=f_classif, k=100)
# # Assuming X_train is your feature matrix (12840 instances x 300 dimensions)
# # and y_train is your target labels

# # Initialize a dictionary to store the indices of top features for each class
# top_features_by_class = {}
# top_scores = {}
# # Calculate the relevance of each feature to each class using chi-squared test
# for label in range(7):  # Assuming you have 7 classes
#     # Create a binary mask indicating instances belonging to the current class
#     mask = (Y_train == label)

#     # SelectKBest with chi2 as the scoring function
#     selector = SelectKBest(score_func=chi2, k=20)  # Select top 20 features
#     selector.fit(X_train_scaled, mask)  # Fit SelectKBest to the data
#     # Get the indices of the top 20 features
#     top_features_indices = np.argsort(selector.scores_)[-20:]
#     scores = selector.scores_[top_features_indices]
#     # Store the indices in the dictionary
#     top_features_by_class[label] = top_features_indices
#     top_scores[label] = scores
    
# # Print the top features for each class
# for label, indices in top_features_by_class.items():
#     print(f"Label {label_decoder[label]}: idx {', '.join(map(str, indices))}")
#     print(top_scores[label])

torch.Size([12840, 102])


In [1]:
# selected_features1[0]

In [2]:
# selected_features2[0]

In [23]:
# pca = PCA(n_components=2)
# pca_result = pca.fit_transform(selected_features.detach().numpy())

# # Plot the PCA result with color-coded labels
# plt.figure(figsize=(8, 6))
# for label in np.unique(Y_train):
#     indices = Y_train == label
#     plt.scatter(pca_result[indices, 0], pca_result[indices, 1], label=f'{label_decoder[label]}', alpha=0.5)
#     plt.title('PCA Visualization of Selected Utterance Embeddings (Train) with Color-Coded Labels')
#     plt.xlabel('Principal Component 1')
#     plt.ylabel('Principal Component 2')
#     plt.legend()
#     plt.grid(True)
#     plt.show()

3d plottly

In [24]:
# X_train = selected_features
# X_train = X_train / np.linalg.norm(X_train, axis=1, keepdims=True)
# # Perform T-SNE dimensionality reduction
# tsne = TSNE(n_components=3, random_state=42)
# X_tsne = tsne.fit_transform(X_train)

# # Create a Plotly scatter plot
# fig = go.Figure(data=[go.Scatter3d(
#     x=X_tsne[:, 0],
#     y=X_tsne[:, 1],
#     z=X_tsne[:, 2],
#     mode='markers',
#     marker=dict(
#         size=3,
#         color=Y_train,  # Assuming Y_train contains labels for coloring
#         colorscale='Viridis',  # You can choose a different colorscale
#         opacity=0.8
#     )
# )])

# # Update layout
# fig.update_layout(title='3D T-SNE Plot', autosize=False,
#                   width=800, height=800)

# # Show the plot
# fig.show()

In [25]:
# Save the plot as an HTML file
# pio.write_html(fig, '3d_tsne_plot.html')

Selected features of train data

In [91]:
# BERT+EGAT
selectedUPrime, BERT_trainIndices = get_selected_features(tensorUtterancesTrain, y_train, 20)
selectedHPrime, GAT_trainIndices1 = get_selected_features(allNodeFeatsTrain, y_train, 20)
concatenatedRepresentationTrain1 = torch.cat((selectedUPrime, selectedHPrime), dim=1)
# BERT+GAT
selectedHPrime, GAT_trainIndices2 = get_selected_features(allNodeFeatsTrain, y_train, 20)
concatenatedRepresentationTrain2 = torch.cat((selectedUPrime, selectedHPrime), dim=1)
# raw-BERT
rawCtxRepresentationTrain = tensorUtterancesTrain
# selected-BERT
ctxRepresentationTrain = selectedUPrime

print("Sizes of different combination of train data\n",
      "FeatureSelected+BERT+EGAT: ", concatenatedRepresentationTrain1.shape, "\n",
      "FeatureSelected+BERT+GAT: ", concatenatedRepresentationTrain2.shape, "\n",
      "BERT: ", rawCtxRepresentationTrain.shape, "\n",
      "FeatureSelected+BERT: ", ctxRepresentationTrain.shape)

Sizes of different combination of train data
 FeatureSelected+BERT+EGAT:  torch.Size([12840, 227]) 
 FeatureSelected+BERT+GAT:  torch.Size([12840, 227]) 
 BERT:  torch.Size([12840, 768]) 
 FeatureSelected+BERT:  torch.Size([12840, 114])


Selected features of test data

In [92]:
# BERT+EGAT
selectedUPrime = tensorUtterancesTest[:, BERT_trainIndices]
selectedHPrime = allNodeFeatsTest[:, GAT_trainIndices1]
concatenatedRepresentationTest1 = torch.cat((selectedUPrime, selectedHPrime), dim=1)
# BERT+GAT
selectedHPrime = allNodeFeatsTest[:, GAT_trainIndices2]
concatenatedRepresentationTest2 = torch.cat((selectedUPrime, selectedHPrime), dim=1)
# raw-BERT
rawCtxRepresentationTest = tensorUtterancesTest
# selected-BERT
ctxRepresentationTest = selectedUPrime

print("Sizes of different combination of train data\n",
      "FeatureSelected+BERT+EGAT: ", concatenatedRepresentationTest1.shape, "\n",
      "FeatureSelected+BERT+GAT: ", concatenatedRepresentationTest2.shape, "\n",
      "BERT: ", rawCtxRepresentationTest.shape, "\n",
      "FeatureSelected+BERT: ", ctxRepresentationTest.shape)

Sizes of different combination of train data
 FeatureSelected+BERT+EGAT:  torch.Size([3400, 227]) 
 FeatureSelected+BERT+GAT:  torch.Size([3400, 227]) 
 BERT:  torch.Size([3400, 768]) 
 FeatureSelected+BERT:  torch.Size([3400, 114])


1. Prep data - normalize and create data loader

In [108]:
def get_data_loader(features, labels, isTrain):
    num_instances = len(features)
    num_classes = 7

    # Rescale input features
    # selected_features = concatenated_representation / np.linalg.norm(concatenated_representation, axis=1, keepdims=True)

    # Apply data resampling (oversampling) to balance class distribution
    if isTrain:
        X_set, Y_set = oversample_data(features, labels, num_classes)
    else:
        X_set, Y_set = features, labels

    # Calculate class weights for class weighting
#     class_counts = np.bincount(labels)
#     total_instances = np.sum(class_counts)
    # class_weights = torch.tensor([total_instances / (num_classes * count) for count in class_counts], dtype=torch.float32)

    # Convert data to PyTorch tensors
    X_tensor = torch.tensor(X_set, dtype=torch.float32).clone().detach()
    Y_tensor = torch.tensor(Y_set, dtype=torch.long).clone().detach()
    # print(X_train_tensor.shape, Y_train_tensor.shape)
    # X_train_tensor = torch.tensor(selected_features)
    # Y_train_tensor = torch.tensor(y_train)

    unique_labels, label_counts = np.unique(Y_set, return_counts=True)

    # Print the counts for each unique label
    for label, count in zip(unique_labels, label_counts):
        print(f"Label {label_decoder[label]}: {count} occurrences")

    print(X_tensor.shape, Y_tensor.shape)
    # Create a TensorDataset
    dataset = TensorDataset(X_tensor, Y_tensor)

    # Define batch size for DataLoader
    batch_size = 1

    # Create a PyTorch DataLoader
    train_loader = DataLoader(dataset, batch_size=batch_size, shuffle=True)

    return train_loader, X_tensor, Y_tensor

2. Training

In [110]:
def model_train(dataLoader, input_dim, output_dim, num_epochs, num_classes):
    # Initialize the model
    model = MyNetwork(input_dim, output_dim, num_classes)
    # Define loss function and optimizer
    criterion = nn.CrossEntropyLoss()
    optimizer = optim.Adam(model.parameters(), lr=0.0001)
    # Train the model
    print_interval = 1  # Print tqdm every 10 epochs
    for epoch in range(num_epochs):
        total_loss = 0.0
        correct_predictions = 0
        total_instances = 0
        for inputs, labels in tqdm(dataLoader, desc=f'Epoch {epoch+1}/{num_epochs}', leave=False):
            # Forward pass
            outputs = model(inputs)
            loss = criterion(outputs, labels)

            # Backward and optimize
            optimizer.zero_grad()
            loss.backward()
            optimizer.step()

            total_loss += loss.item() * inputs.size(0)
            _, predicted = torch.max(outputs, 1)
            correct_predictions += (predicted == labels).sum().item()
            total_instances += labels.size(0)

        # Print average loss and accuracy per epoch
        if (epoch + 1) % print_interval == 0:
            epoch_loss = total_loss / len(dataLoader.dataset)
            epoch_accuracy = correct_predictions / total_instances
            print(f'Epoch [{epoch+1}/{num_epochs}], Loss: {epoch_loss:.4f}, Accuracy: {epoch_accuracy:.4f}')

    return model

In [117]:
def classify_emotions(model, X_tensor, Y_tensor, isTrain):
    # Set the model to evaluation mode
    model.eval()

    # Predict on the training data
    with torch.no_grad():
        outputs = model(X_tensor)
        _, predicted = torch.max(outputs, 1)

    # Convert predicted tensor to numpy array
    predicted = predicted.numpy()

    # Calculate F1 score per class
    f1_per_class = f1_score(Y_tensor, predicted, average=None)
    f1 = f1_score(Y_tensor, predicted, average='macro')
    if isTrain:
        print(f'Train F1 Score: {f1:.4f}')
    else:
        print(f'Test F1 Score: {f1:.4f}')

    unique_labels, label_counts = np.unique(predicted, return_counts=True)

    # Print F1 score for each class
    for i, f1 in enumerate(f1_per_class):
        print(f'F1 Score for Class {label_decoder[i]}: {f1:.4f}')

    # Print the counts for each unique label
    for label, count in zip(unique_labels, label_counts):
        print(f"Label {label_decoder[label]}: {count} occurrences")

In [ ]:
# TODO create table of loss and accuracy during training
# TODO also compute the time it takes to complete the train

<h4> Train and validate BERT+EGAT given 2 epochs

In [112]:
dataLoader, X_trainTensor, Y_trainTensor = get_data_loader(concatenatedRepresentationTrain1, y_train, True)
# TODO 3rd argument is tunable
fcClf = model_train(dataLoader, 
                    input_dim=X_trainTensor.shape[1], 
                    output_dim=20, 
                    num_epochs=2, 
                    num_classes=7)
classify_emotions(fcClf, X_trainTensor, Y_trainTensor, True)

Label anger: 5960 occurrences
Label disgust: 5960 occurrences
Label fear: 5960 occurrences
Label joy: 5960 occurrences
Label neutral: 5960 occurrences
Label sadness: 5960 occurrences
Label surprise: 5960 occurrences
torch.Size([41720, 227]) torch.Size([41720])


Epoch [1/2], Loss: 1.8432, Accuracy: 0.3098


Epoch [2/2], Loss: 1.7574, Accuracy: 0.3779
Training F1 Score: 0.3981
F1 Score for Class anger: 0.2814
F1 Score for Class disgust: 0.4454
F1 Score for Class fear: 0.4588
F1 Score for Class joy: 0.4180
F1 Score for Class neutral: 0.3400
F1 Score for Class sadness: 0.3426
F1 Score for Class surprise: 0.5007
Label anger: 4673 occurrences
Label disgust: 6756 occurrences
Label fear: 5460 occurrences
Label joy: 4867 occurrences
Label neutral: 7234 occurrences
Label sadness: 5325 occurrences
Label surprise: 7405 occurrences


In [113]:
# Predict the test set
_, X_testTensor, Y_testTensor = get_data_loader(concatenatedRepresentationTest1, y_test, False)
classify_emotions(fcClf, X_testTensor, Y_testTensor, False)

Label anger: 516 occurrences
Label disgust: 99 occurrences
Label fear: 60 occurrences
Label joy: 495 occurrences
Label neutral: 1615 occurrences
Label sadness: 263 occurrences
Label surprise: 352 occurrences
torch.Size([3400, 227]) torch.Size([3400])
Training F1 Score: 0.1284
F1 Score for Class anger: 0.1551
F1 Score for Class disgust: 0.0665
F1 Score for Class fear: 0.0112
F1 Score for Class joy: 0.1614
F1 Score for Class neutral: 0.3280
F1 Score for Class sadness: 0.0774
F1 Score for Class surprise: 0.0992
Label anger: 348 occurrences
Label disgust: 412 occurrences
Label fear: 297 occurrences
Label joy: 558 occurrences
Label neutral: 873 occurrences
Label sadness: 357 occurrences
Label surprise: 555 occurrences


C:\Users\edayo\AppData\Local\Temp\ipykernel_1452\2284308149.py:20: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  X_tensor = torch.tensor(X_set, dtype=torch.float32).clone().detach()
C:\Users\edayo\AppData\Local\Temp\ipykernel_1452\2284308149.py:21: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  Y_tensor = torch.tensor(Y_set, dtype=torch.long).clone().detach()


<h4> Train and validate BERT+GAT given 2 epoch

In [114]:
dataLoader, X_trainTensor, Y_trainTensor = get_data_loader(concatenatedRepresentationTrain2, y_train, True)
# TODO 3rd argument is tunable
fcClf = model_train(dataLoader, 
                    input_dim=X_trainTensor.shape[1], 
                    output_dim=20, 
                    num_epochs=2, 
                    num_classes=7)
classify_emotions(fcClf, X_trainTensor, Y_trainTensor, True)

Label anger: 5960 occurrences
Label disgust: 5960 occurrences
Label fear: 5960 occurrences
Label joy: 5960 occurrences
Label neutral: 5960 occurrences
Label sadness: 5960 occurrences
Label surprise: 5960 occurrences
torch.Size([41720, 227]) torch.Size([41720])


Epoch [1/2], Loss: 1.8432, Accuracy: 0.3084


Epoch [2/2], Loss: 1.7602, Accuracy: 0.3755
Training F1 Score: 0.3780
F1 Score for Class anger: 0.2380
F1 Score for Class disgust: 0.4426
F1 Score for Class fear: 0.4328
F1 Score for Class joy: 0.4494
F1 Score for Class neutral: 0.3082
F1 Score for Class sadness: 0.2854
F1 Score for Class surprise: 0.4893
Label anger: 3033 occurrences
Label disgust: 6110 occurrences
Label fear: 5750 occurrences
Label joy: 7529 occurrences
Label neutral: 5707 occurrences
Label sadness: 3823 occurrences
Label surprise: 9768 occurrences


In [115]:
# Predict the test set
_, X_testTensor, Y_testTensor = get_data_loader(concatenatedRepresentationTest2, y_test, False)
classify_emotions(fcClf, X_testTensor, Y_testTensor, False)

Label anger: 516 occurrences
Label disgust: 99 occurrences
Label fear: 60 occurrences
Label joy: 495 occurrences
Label neutral: 1615 occurrences
Label sadness: 263 occurrences
Label surprise: 352 occurrences
torch.Size([3400, 227]) torch.Size([3400])
Training F1 Score: 0.1230
F1 Score for Class anger: 0.1178
F1 Score for Class disgust: 0.0674
F1 Score for Class fear: 0.0110
F1 Score for Class joy: 0.1861
F1 Score for Class neutral: 0.2777
F1 Score for Class sadness: 0.0696
F1 Score for Class surprise: 0.1314
Label anger: 231 occurrences
Label disgust: 346 occurrences
Label fear: 304 occurrences
Label joy: 816 occurrences
Label neutral: 675 occurrences
Label sadness: 254 occurrences
Label surprise: 774 occurrences


C:\Users\edayo\AppData\Local\Temp\ipykernel_1452\2284308149.py:20: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  X_tensor = torch.tensor(X_set, dtype=torch.float32).clone().detach()
C:\Users\edayo\AppData\Local\Temp\ipykernel_1452\2284308149.py:21: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  Y_tensor = torch.tensor(Y_set, dtype=torch.long).clone().detach()


<h4> Train and validate BERT (x feature selection) given 2 epochs

In [126]:
dataLoader, X_trainTensor, Y_trainTensor = get_data_loader(rawCtxRepresentationTrain, y_train, True)
# TODO 3rd argument is tunable
fcClf = model_train(dataLoader, 
                    input_dim=X_trainTensor.shape[1], 
                    output_dim=50, 
                    num_epochs=2, 
                    num_classes=7)
classify_emotions(fcClf, X_trainTensor, Y_trainTensor, True)

Label anger: 5960 occurrences
Label disgust: 5960 occurrences
Label fear: 5960 occurrences
Label joy: 5960 occurrences
Label neutral: 5960 occurrences
Label sadness: 5960 occurrences
Label surprise: 5960 occurrences
torch.Size([41720, 768]) torch.Size([41720])


Epoch [1/2], Loss: 1.7511, Accuracy: 0.3875


Epoch [2/2], Loss: 1.6316, Accuracy: 0.4807
Train F1 Score: 0.4958
F1 Score for Class anger: 0.3838
F1 Score for Class disgust: 0.6268
F1 Score for Class fear: 0.6158
F1 Score for Class joy: 0.4928
F1 Score for Class neutral: 0.3800
F1 Score for Class sadness: 0.3978
F1 Score for Class surprise: 0.5736
Label anger: 5514 occurrences
Label disgust: 9940 occurrences
Label fear: 5856 occurrences
Label joy: 5298 occurrences
Label neutral: 5402 occurrences
Label sadness: 3553 occurrences
Label surprise: 6157 occurrences


In [127]:
# Predict the test set
_, X_testTensor, Y_testTensor = get_data_loader(rawCtxRepresentationTest, y_test, False)
classify_emotions(fcClf, X_testTensor, Y_testTensor, False)

Label anger: 516 occurrences
Label disgust: 99 occurrences
Label fear: 60 occurrences
Label joy: 495 occurrences
Label neutral: 1615 occurrences
Label sadness: 263 occurrences
Label surprise: 352 occurrences
torch.Size([3400, 768]) torch.Size([3400])
Test F1 Score: 0.1247
F1 Score for Class anger: 0.1617
F1 Score for Class disgust: 0.0646
F1 Score for Class fear: 0.0060
F1 Score for Class joy: 0.1737
F1 Score for Class neutral: 0.3011
F1 Score for Class sadness: 0.0607
F1 Score for Class surprise: 0.1049
Label anger: 461 occurrences
Label disgust: 551 occurrences
Label fear: 275 occurrences
Label joy: 645 occurrences
Label neutral: 736 occurrences
Label sadness: 264 occurrences
Label surprise: 468 occurrences


C:\Users\edayo\AppData\Local\Temp\ipykernel_1452\2284308149.py:20: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  X_tensor = torch.tensor(X_set, dtype=torch.float32).clone().detach()
C:\Users\edayo\AppData\Local\Temp\ipykernel_1452\2284308149.py:21: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  Y_tensor = torch.tensor(Y_set, dtype=torch.long).clone().detach()


<h4> Train and validate BERT (o feature selection) given 2 epochs

In [129]:
dataLoader, X_trainTensor, Y_trainTensor = get_data_loader(ctxRepresentationTrain, y_train, True)
# TODO 3rd argument is tunable
fcClf = model_train(dataLoader, 
                    input_dim=X_trainTensor.shape[1], 
                    output_dim=50, 
                    num_epochs=2, 
                    num_classes=7)
classify_emotions(fcClf, X_trainTensor, Y_trainTensor, True)

Label anger: 5960 occurrences
Label disgust: 5960 occurrences
Label fear: 5960 occurrences
Label joy: 5960 occurrences
Label neutral: 5960 occurrences
Label sadness: 5960 occurrences
Label surprise: 5960 occurrences
torch.Size([41720, 114]) torch.Size([41720])


Epoch [1/2], Loss: 1.8229, Accuracy: 0.3227


Epoch [2/2], Loss: 1.7423, Accuracy: 0.3817
Train F1 Score: 0.3853
F1 Score for Class anger: 0.2409
F1 Score for Class disgust: 0.4296
F1 Score for Class fear: 0.4482
F1 Score for Class joy: 0.4354
F1 Score for Class neutral: 0.3468
F1 Score for Class sadness: 0.3023
F1 Score for Class surprise: 0.4939
Label anger: 3878 occurrences
Label disgust: 7479 occurrences
Label fear: 6252 occurrences
Label joy: 4900 occurrences
Label neutral: 7142 occurrences
Label sadness: 4500 occurrences
Label surprise: 7569 occurrences


In [130]:
# Predict the test set
_, X_testTensor, Y_testTensor = get_data_loader(ctxRepresentationTest, y_test, False)
classify_emotions(fcClf, X_testTensor, Y_testTensor, False)

Label anger: 516 occurrences
Label disgust: 99 occurrences
Label fear: 60 occurrences
Label joy: 495 occurrences
Label neutral: 1615 occurrences
Label sadness: 263 occurrences
Label surprise: 352 occurrences
torch.Size([3400, 114]) torch.Size([3400])
Test F1 Score: 0.1248
F1 Score for Class anger: 0.1312
F1 Score for Class disgust: 0.0577
F1 Score for Class fear: 0.0098
F1 Score for Class joy: 0.1586
F1 Score for Class neutral: 0.3388
F1 Score for Class sadness: 0.0704
F1 Score for Class surprise: 0.1070
Label anger: 292 occurrences
Label disgust: 421 occurrences
Label fear: 349 occurrences
Label joy: 564 occurrences
Label neutral: 876 occurrences
Label sadness: 334 occurrences
Label surprise: 564 occurrences


C:\Users\edayo\AppData\Local\Temp\ipykernel_1452\2284308149.py:20: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  X_tensor = torch.tensor(X_set, dtype=torch.float32).clone().detach()
C:\Users\edayo\AppData\Local\Temp\ipykernel_1452\2284308149.py:21: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  Y_tensor = torch.tensor(Y_set, dtype=torch.long).clone().detach()
